#### Test mynds packages in the sandbox

In [ ]:
from collections.abc import Mapping
from functools import partial
from pathlib import Path
from typing import Callable, TypeVar

import tqdm

from mynd.camera import CameraID
from mynd.collections import CameraGroup
from mynd.image import (
    Image,
    ImageType,
    ImageComposite,
    ImageCompositeLoader,
)
from mynd.io import read_image

from mynd.utils.composition import create_composite_builder
from mynd.utils.filesystem import (
    list_directory,
    create_resource,
    Resource,
    ResourceManager,
    create_resource_matcher,
)
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


IMAGE_PATTERN: str = "*.tiff"


def create_mockup_cameras() -> CameraGroup:
    """Creates a mockup of cameras with corresponding sensors."""
    
    cameras: list[CameraID] = [
        CameraID(0, "PR_20100605_030049_596_LC16"),
        CameraID(1, "PR_20100605_030049_596_RM16"),
        CameraID(2, "PR_20100605_030050_576_LC16"),
        CameraID(3, "PR_20100605_030050_576_RM16"),
        CameraID(4, "PR_20100605_030051_609_LC16"),
        CameraID(5, "PR_20100605_030051_609_RM16"),
    ]

    sensors: list[SensorID] = [
        SensorID(0, "stereo_left"),
        SensorID(1, "stereo_right"),
    ]

    cameras_to_sensors: dict[CameraID, SensorID] = {
        cameras[0]: sensors[0],
        cameras[1]: sensors[1],
        cameras[2]: sensors[0],
        cameras[3]: sensors[1],
        cameras[4]: sensors[0],
        cameras[5]: sensors[1],
    }
    
    raise NotImplementedError


def main() -> None:
    """Main entrypoint."""

    DATADIR: Path = Path("/home/martin/data")
    
    
    directories: dict[str, Path] = {
        ImageType.COLOR: DATADIR
        / Path("acfr_images_debayered/r23685bc_20100605_021022_debayered"),
        ImageType.RANGE: DATADIR
        / Path("acfr_stereo_ranges/r23685bc_20100605_021022_ranges"),
        ImageType.NORMAL: DATADIR
        / Path("acfr_stereo_normals/r23685bc_20100605_021022_normals"),
    }

    tags: dict[ImageType, list[str]] = {
        ImageType.COLOR: ["image", "color"],
        ImageType.RANGE: ["image", "range"],
        ImageType.NORMAL: ["image", "normal"],
    }

    image_types: list[ImageType] = list(directories.keys())

    # Step 1 - Collect image resources
    image_manager: ResourceManager = collect_image_resources(directories, tags)

    # Step 2 - Retrieve image resources by type
    query_results: dict[ImageType, list[Resource]] = {
        image_type: image_manager.query_tags(tags.get(image_type))
        for image_type in image_types
    }

    log_image_queries(query_results)

    # Step 3 - Match image resources to create composites
    matcher = create_resource_matcher()
    matches: list[ComponentMatch] = matcher(query_results)

    # Step 4 - Create loaders from the matched resources
    builder = create_composite_builder(
        factory=create_image_composite_loader,
        labeller=label_image_composite,
    )

    image_loaders: dict[str, ImageCompositeLoader] = builder(matches)

    # Map image loaders to cameras
    filtered_loaders: dict[CameraID, ImageCompositeLoader] = {
        camera: image_loaders.get(camera) for camera in cameras 
        if camera.label in image_loaders
    }

    # TODO: Map loaders to cameras
    logger.info(f"Cameras: {len(cameras)}")
    logger.info(f"Loaders: {len(image_loaders)}")
    logger.info(f"Filtered loaders: {len(filtered_loaders)}")


# INVOKE MAIN
main()